In [7]:
# My API key :350143be5c876ffb78a910536d4089f7
# Register on OpenWeatherMap and generate your API key
# For activating API key you need subscription this code will not work without valid api 

In [22]:
# Weather Data Analysis from OpenWeatherMap API
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------------
# 1. Set API key and location
# -----------------------------
api_key = "103356cae2328889ac6515ebab51266f"  # Replace with your own key
city = "Pune,IN"  # City,Country
url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}"

# -----------------------------
# 2. Send API request
# -----------------------------
response = requests.get(url)
weather_data = response.json()

# -----------------------------
# 3. Check if request was successful
# -----------------------------
if weather_data.get('cod') != "200":
    print("API request failed:", weather_data.get('message'))
else:
    print("API request successful!")

    # -----------------------------
    # 4. Extract weather attributes
    # -----------------------------
    forecast_list = weather_data['list']

    timestamps = [pd.to_datetime(item['dt'], unit='s') for item in forecast_list]
    temperature = [item['main']['temp'] - 273.15 for item in forecast_list]  # Kelvin to Celsius
    humidity = [item['main']['humidity'] for item in forecast_list]
    wind_speed = [item['wind']['speed'] for item in forecast_list]
    precipitation = [item.get('rain', {}).get('3h', 0) for item in forecast_list]  # Rain in last 3 hours
    description = [item['weather'][0]['description'] for item in forecast_list]

    # -----------------------------
    # 5. Create DataFrame
    # -----------------------------
    weather_df = pd.DataFrame({
        'Timestamp': timestamps,
        'Temperature (°C)': temperature,
        'Humidity (%)': humidity,
        'Wind Speed (m/s)': wind_speed,
        'Precipitation (mm)': precipitation,
        'Description': description
    })
    weather_df.set_index('Timestamp', inplace=True)

    # -----------------------------
    # 6. Summary statistics
    # -----------------------------
    print("Summary Statistics:")
    print(weather_df.describe())

    # -----------------------------
    # 7. Visualizations
    # -----------------------------
    plt.figure(figsize=(12,6))
    plt.plot(weather_df.index, weather_df['Temperature (°C)'], marker='o', color='orange')
    plt.title("Temperature Trend Over Time")
    plt.xlabel("Timestamp")
    plt.ylabel("Temperature (°C)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(12,6))
    plt.plot(weather_df.index, weather_df['Humidity (%)'], marker='o', color='blue')
    plt.title("Humidity Trend Over Time")
    plt.xlabel("Timestamp")
    plt.ylabel("Humidity (%)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(12,6))
    plt.bar(weather_df.index, weather_df['Precipitation (mm)'], color='skyblue')
    plt.title("Precipitation Over Time")
    plt.xlabel("Timestamp")
    plt.ylabel("Precipitation (mm)")
    plt.xticks(rotation=45)
    plt.show()

    plt.figure(figsize=(8,6))
    sns.heatmap(weather_df[['Temperature (°C)', 'Humidity (%)', 'Wind Speed (m/s)', 'Precipitation (mm)']].corr(),
                annot=True, cmap='coolwarm')
    plt.title("Correlation Between Weather Attributes")
    plt.show()

    # -----------------------------
    # 8. Daily aggregation
    # -----------------------------
    daily_summary = weather_df.resample('D').agg({
        'Temperature (°C)': ['mean', 'max', 'min'],
        'Humidity (%)': 'mean',
        'Wind Speed (m/s)': 'mean',
        'Precipitation (mm)': 'sum'
    })
    print("Daily Summary:")
    print(daily_summary)


API request failed: Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.
